In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_ta as ta
import re
import requests

from datetime import datetime

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [2]:
from invaas.schwab.schwab_task import SchwabTask

task = SchwabTask(env="local")
await task.setup_api()

2024-01-29 22:05:29 WARNING No DBUtils defined in the runtime
2024-01-29 22:05:29 INFO Initializing task for local environment
2024-01-29 22:05:29 INFO Preparing to trade QQQ
2024-01-29 22:05:29 INFO Logging in to Schwab


In [3]:
def get_options_chains_data(ticker: str):
    options_info = requests.get(f"https://client.schwab.com/symlup/OptionsCwp/Underlying/{ticker}.txt").text
    options_info = re.sub(r"[\n\t\r]*", "", options_info)
    options_info = json.loads(options_info.replace('SuggestionBox.JsonpCallback("option",', "")[:-1])
    expirations = [x["Date"] for x in options_info["Expirations"]]
    options_chain_url = f"https://ausgateway.schwab.com/api/is.CSOptionChainsWeb/v1/OptionChainsPort/OptionChains/chains?Symbol={ticker}&IncludeGreeks=false&{','.join(expirations)}"
    task.schwab_api.update_token("update")
    return json.loads(requests.get(options_chain_url, headers=task.schwab_api.headers).text)


options_chains_data = get_options_chains_data("QQQ")
options_chains_data

{'UnderlyingData': {'Ask': '428.69',
  'AskSize': '6',
  'Bid': '428.65',
  'BidSize': '6',
  'Close': '423.81',
  'CumulativeVolume': '38694724',
  'Description': 'INVSC QQQ TRUST SRS 1',
  'ErrorCode': 0,
  'InstrumentType': 'ETF',
  'Last': '428.15',
  'NetChange': '4.34',
  'PercentChange': '1.02',
  'Open': '424.27',
  'DailyHigh': '428.35',
  'DailyLow': '423.61',
  'High52W': '429.85',
  'Low52W': '285.19',
  'Symbol': 'QQQ',
  'DividendYield': 0.5,
  'ExchangeName': 'NASDAQ',
  'IsDelayed': False},
 'Expirations': [{'ExpirationGroup': {'RootSymbol': 'QQQ',
    'MonthAndDay': 'Jan 29',
    'Year': '2024',
    'Day': 'Mon',
    'SettlementType': '',
    'DaysUntil': 0},
   'AdjustedLabel': {},
   'AdjustedGroup': '',
   'Chains': [{'SymbolGroup': 'QQQ20240129353',
     'Legs': [{'Sym': 'QQQ   240129C00353000',
       'OptionType': 'C',
       'Strk': '353',
       'Chg': '0.55',
       'ChgPct': '0.76',
       'Bid': '73.73',
       'Ask': '76.06',
       'Lst': '72.22',
       '

In [4]:
data = []
underlying_last = float(options_chains_data["UnderlyingData"]["Last"])

for expiration in options_chains_data["Expirations"]:
    days_until_expiration = expiration["ExpirationGroup"]["DaysUntil"]
    expiration_month_day = expiration["ExpirationGroup"]["MonthAndDay"].split(" ")
    expiration_month = expiration_month_day[0]
    expiration_day = expiration_month_day[1]
    expiration_year = expiration["ExpirationGroup"]["Year"]
    expiration_date = datetime.strptime(f"{expiration_month} {expiration_day} {expiration_year}", "%b %d %Y")

    for chain in expiration["Chains"]:
        data_obj = {}
        data_obj["OPTION_ID"] = chain["SymbolGroup"]
        data_obj["UNDERLYING_LAST"] = underlying_last
        data_obj["EXPIRE_DATE"] = expiration_date
        data_obj["DTE"] = days_until_expiration

        for leg in chain["Legs"]:
            data_obj["STRIKE"] = float(leg["Strk"])
            data_obj["STRIKE_DISTANCE"] = abs(data_obj["STRIKE"] - underlying_last)
            data_obj["STRIKE_DISTANCE_PCT"] = data_obj["STRIKE_DISTANCE"] / underlying_last
            leg_prefix = leg["OptionType"] + "_"
            data_obj[leg_prefix + "ID"] = leg["Sym"]
            data_obj[leg_prefix + "BID"] = float(leg["Bid"])
            data_obj[leg_prefix + "ASK"] = float(leg["Ask"])
            data_obj[leg_prefix + "VOLUME"] = int(leg["Vol"])

        data.append(data_obj)

df_options_chains = pd.DataFrame(data=data)
df_options_chains

,OPTION_ID,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,C_ID,C_BID,C_ASK,C_VOLUME,P_ID,P_BID,P_ASK,P_VOLUME
0,QQQ20240129353,428.15,2024-01-29,0,353.0,75.15,0.175523,QQQ 240129C00353000,73.73,76.06,18,QQQ 240129P00353000,0.00,0.01,5
1,QQQ20240129354,428.15,2024-01-29,0,354.0,74.15,0.173187,QQQ 240129C00354000,72.73,75.06,0,QQQ 240129P00354000,0.00,0.01,0
2,QQQ20240129355,428.15,2024-01-29,0,355.0,73.15,0.170851,QQQ 240129C00355000,71.73,74.06,14,QQQ 240129P00355000,0.00,0.01,0
3,QQQ20240129356,428.15,2024-01-29,0,356.0,72.15,0.168516,QQQ 240129C00356000,70.73,73.06,0,QQQ 240129P00356000,0.00,0.01,0
4,QQQ20240129357,428.15,2024-01-29,0,357.0,71.15,0.166180,QQQ 240129C00357000,69.73,72.06,0,QQQ 240129P00357000,0.00,0.01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4183,QQQ20261218620,428.15,2026-12-18,1054,620.0,191.85,0.448091,QQQ 261218C00620000,11.00,16.00,0,QQQ 261218P00620000,189.55,194.50,0
4184,QQQ20261218625,428.15,2026-12-18,1054,625.0,196.85,0.459769,QQQ 261218C00625000,10.50,15.00,0,QQQ 261218P00625000,194.55,199.50,0
4185,QQQ20261218630,428.15,2026-12-18,1054,630.0,201.85,0.471447,QQQ 261218C00630000,9.50,14.50,0,QQQ 261218P00630000,199.55,204.50,0
4186,QQQ20261218635,428.15,2026-12-18,1054,635.0,206.85,0.483125,QQQ 261218C00635000,10.77,13.50,120,QQQ 261218P00635000,204.55,209.50,0


In [15]:
from invaas.schwab.cnn_fear_greed_index import get_historical_cnn_fear_greed_index


def get_fear_greed_index_data():
    historical_period = 14
    df = pd.DataFrame(data=get_historical_cnn_fear_greed_index()["data"])
    df.set_index(
        pd.DatetimeIndex([pd.Timestamp(x, unit="ms", tz="UTC") for x in df.x]),
        inplace=True,
    )
    df.rename(columns={"y": "fear_greed_index"}, inplace=True)
    df.sort_index(inplace=True)
    df["fear_greed_index"] = df.fear_greed_index.fillna(method="ffill").astype(float)
    df["previous_max_fear_greed_index"] = (
        df["fear_greed_index"].rolling(window=historical_period, min_periods=historical_period).max()
    )
    df["previous_min_fear_greed_index"] = (
        df["fear_greed_index"].rolling(window=historical_period, min_periods=historical_period).min()
    )
    return (
        int(df.fear_greed_index.values[-1]),
        int(df.previous_max_fear_greed_index.values[-1]),
        int(df.previous_min_fear_greed_index.values[-1]),
    )


current_fear_greed_index, previous_max_fear_greed_index, previous_min_fear_greed_index = get_fear_greed_index_data()

print(f"Current fear greed index: {current_fear_greed_index}")
print(f"Previous max fear greed index: {previous_max_fear_greed_index}")
print(f"Previous min fear greed index: {previous_min_fear_greed_index}")
print()

good_call_buy = current_fear_greed_index >= previous_max_fear_greed_index - 2
good_put_buy = current_fear_greed_index <= previous_min_fear_greed_index + 2

print(f"Buy call: {good_call_buy}")
print(f"Buy put: {good_put_buy}")

Current fear greed index: 76
Previous max fear greed index: 77
Previous min fear greed index: 61

Buy call: True
Buy put: False


In [14]:
min_dte = 14
min_volume = 100
max_strike_distance_pct = 0.1
max_buy_price = 100

df_options_to_buy = df_options_chains.loc[
    (df_options_chains.DTE > min_dte)
    & (df_options_chains.STRIKE_DISTANCE_PCT < max_strike_distance_pct)
    & (
        (good_call_buy & (df_options_chains.C_VOLUME > min_volume) & (df_options_chains.C_ASK * 100 < max_buy_price))
        | (good_put_buy & (df_options_chains.P_VOLUME > min_volume) & (df_options_chains.P_ASK * 100 < max_buy_price))
    )
].sort_values(by="STRIKE_DISTANCE_PCT")
df_options_to_buy

,OPTION_ID,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,C_ID,C_BID,C_ASK,C_VOLUME,P_ID,P_BID,P_ASK,P_VOLUME
1516,QQQ20240216449.78,428.15,2024-02-16,18,449.78,21.63,0.050520,QQQ 240216C00449780,0.69,0.71,324,QQQ 240216P00449780,21.80,22.07,0
1517,QQQ20240216450,428.15,2024-02-16,18,450.00,21.85,0.051034,QQQ 240216C00450000,0.68,0.69,6275,QQQ 240216P00450000,22.08,22.23,6
1519,QQQ20240216455,428.15,2024-02-16,18,455.00,26.85,0.062712,QQQ 240216C00455000,0.35,0.36,4688,QQQ 240216P00455000,26.92,27.25,0
1723,QQQ20240301455,428.15,2024-03-01,32,455.00,26.85,0.062712,QQQ 240301C00455000,0.96,0.98,115,QQQ 240301P00455000,26.92,27.24,0
1628,QQQ20240223460,428.15,2024-02-23,25,460.00,31.85,0.074390,QQQ 240223C00460000,0.33,0.34,842,QQQ 240223P00460000,31.92,32.24,9
1725,QQQ20240301465,428.15,2024-03-01,32,465.00,36.85,0.086068,QQQ 240301C00465000,0.34,0.37,331,QQQ 240301P00465000,36.92,37.24,0
2028,QQQ20240315465,428.15,2024-03-15,46,465.00,36.85,0.086068,QQQ 240315C00465000,0.84,0.85,374,QQQ 240315P00465000,36.92,37.24,0
2029,QQQ20240315469.78,428.15,2024-03-15,46,469.78,41.63,0.097232,QQQ 240315C00469780,0.55,0.57,311,QQQ 240315P00469780,41.70,42.02,0
2030,QQQ20240315470,428.15,2024-03-15,46,470.00,41.85,0.097746,QQQ 240315C00470000,0.54,0.56,123,QQQ 240315P00470000,41.92,42.24,0


In [13]:
positions = task.schwab_api.get_balance_positions()
owned_option_positions = [
    x for x in positions["positionDetails"]["positions"] if x["securityType"] == "Option" and x["shares"] > 0
]
owned_call_options = [x for x in owned_option_positions if x["symbolDescription"].startswith("CALL")]
owned_put_options = [x for x in owned_option_positions if x["symbolDescription"].startswith("PUT")]
print(owned_call_options)
print(owned_put_options)
positions

[]
[]


{'brokerageAccountId': 47004171,
 'accountType': 0,
 'dayTradeCount': 1,
 'isPatternDayTrader': False,
 'pendingPurchasesEnabled': True,
 'showPendingPurchasesBanner': False,
 'openOrdersIncluded': True,
 'balanceDetails': {'availableToTradeBalances': {'cash': 504.79,
   'settledFunds': 504.79,
   'cashWithBorrow': 504.79,
   'sma': 47.0,
   'netWorth': 599.35},
  'marginableSecurityBalances': {'equitiesAndEtfs': 504.79,
   'mutualFunds': 504.79,
   'shortSelling': 504.79},
  'nonMarginableSecurityBalances': {'equitiesAndEtfs': 504.79,
   'mutualFunds': 504.79,
   'pennyStocks': 504.79},
  'optionsBalances': {'longOptions': 504.79, 'shortOptions': 555.43}},
 'positionDetails': {'positions': [{'symbol': 'AAPL',
    'displaySymbol': None,
    'symbolDescription': 'APPLE INC',
    'securityType': 'Stock/ETF',
    'itemIssueId': 1973757747,
    'shares': 0.0973,
    'shortShares': 0.0,
    'totalShares': 0.0973,
    'totalOptions': 0.0,
    'reinvestDividend': True,
    'reinvestCapitalGai

In [8]:
df_owned_options_chains = df_options_chains.loc[
    (df_options_chains.C_ID.isin(owned_call_options)) | (df_options_chains.P_ID.isin(owned_put_options))
]
df_owned_options_chains

,OPTION_ID,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,C_ID,C_BID,C_ASK,C_VOLUME,P_ID,P_BID,P_ASK,P_VOLUME


In [9]:
task.schwab_api.get_balance_positions()
# task.schwab_api.trade(
#     ticker=df_owned_options_chains.C_ID.values[0], asset_class="Option", side="Sell", qty=1, dry_run=False
# )

{'brokerageAccountId': 47004171,
 'accountType': 0,
 'dayTradeCount': 1,
 'isPatternDayTrader': False,
 'pendingPurchasesEnabled': True,
 'showPendingPurchasesBanner': False,
 'openOrdersIncluded': True,
 'balanceDetails': {'availableToTradeBalances': {'cash': 504.79,
   'settledFunds': 504.79,
   'cashWithBorrow': 504.79,
   'sma': 47.0,
   'netWorth': 599.35},
  'marginableSecurityBalances': {'equitiesAndEtfs': 504.79,
   'mutualFunds': 504.79,
   'shortSelling': 504.79},
  'nonMarginableSecurityBalances': {'equitiesAndEtfs': 504.79,
   'mutualFunds': 504.79,
   'pennyStocks': 504.79},
  'optionsBalances': {'longOptions': 504.79, 'shortOptions': 555.43}},
 'positionDetails': {'positions': [{'symbol': 'AAPL',
    'displaySymbol': None,
    'symbolDescription': 'APPLE INC',
    'securityType': 'Stock/ETF',
    'itemIssueId': 1973757747,
    'shares': 0.0973,
    'shortShares': 0.0,
    'totalShares': 0.0973,
    'totalOptions': 0.0,
    'reinvestDividend': True,
    'reinvestCapitalGai

In [10]:
task.schwab_api.get_transaction_history()

{'fromDate': '01/29/2020',
 'toDate': '01/29/2024',
 'totalTransactionsAmount': '-$1,495.71',
 'brokerageTransactions': [{'transactionDate': '01/29/2024',
   'action': 'Sell to Close',
   'symbol': 'QQQ 03/01/2024 465.00 C',
   'description': 'CALL INVESCO QQQ TR $465 EXP 03/01/24',
   'executionPrice': '$0.25',
   'shareQuantity': '1',
   'feesAndCommission': '$0.66',
   'amount': '$24.34',
   'sourceCode': '',
   'effectiveDate': '01/29/2024',
   'depositSequenceId': '0',
   'checkNumber': '',
   'checkDate': '01/30/2024',
   'showWireDetailsLink': False,
   'showCheckDetailsLink': False,
   'showDepositDetailsLink': False,
   'showTradeDetailsLink': True,
   'showAdvisoryProgramCreditDetailsLink': False,
   'transactionDetail': {'cusipId': '',
    'securityNumber': '4078115',
    'accruedInterest': '',
    'redemptionFee': '',
    'stateTaxes': '',
    'withHoldingTaxes': '',
    'otherAmount': '',
    'commission': '$0.65',
    'primeBrokerAmount': '',
    'principal': '$25.00',
  